# Первая задача

In [ ]:
import json
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
import re
import json
import numpy as np


def parse_file(f_name):
    with open(f_name, encoding="utf-8") as f:
        text = ""
        for line in f.readlines():
            text += line

        site = BeautifulSoup(text, "html.parser")

        item = dict()
        item["type"] = site.find_all("span", string=re.compile("Тип:"))[0].get_text().replace("Тип:", "").strip()
        item["title"] = site.find_all("h1")[0].get_text().replace("Турнир:", "").strip()

        address = site.find_all("p", attrs={"class": "address-p"})[0].get_text().split("Начало:")
        item["city"] = address[0].split()[-1].strip()
        item["date_start"] = address[1].strip()

        item["count"] = int(site.find_all("span", attrs={"class": "count"})[0]
                            .get_text().split("Количество туров:")[-1].strip())
        item["time_control"] = site.find_all("span", attrs={"class": "year"})[0] \
            .get_text().split("Контроль времени:")[-1].strip()
        item["min_rating"] = int(site.find_all("span", string=re.compile("Минимальный рейтинг для участия:"))[0]
                                 .get_text().split("Минимальный рейтинг для участия:")[-1].strip())
        item["img"] = site.find_all("img")[0]["src"]
        item["rating"] = float(site.find_all("span", string=re.compile("Рейтинг:"))[0]
                               .get_text().split("Рейтинг:")[-1].strip())
        item["views"] = int(site.find_all("span", string=re.compile("Просмотры:"))[0]
                            .get_text().split("Просмотры:")[-1].strip())
    return item


def calc_stats(col, items_lst):
    numbers = list(map(lambda x: x[col], items_lst))
    numeric_stats = {'sum': sum(numbers),
                     'avg': round(np.average(numbers), 2),
                     'min': min(numbers),
                     'max': max(numbers),
                     'std': round(np.std(numbers), 2),
                     }

    return numeric_stats


def calc_frequency(col, items_lst):
    frequency = {}

    for item in items_lst:
        frequency[item[col]] = frequency.get(item[col], 0) + 1

    return frequency


items = []
for i in range(1, 1000):
    file_name = f"/content/Данные/{i}.html"
    result = parse_file(file_name)
    items.append(result)

items = sorted(items, key=lambda x: x["views"], reverse=True)

with open("results/task1_all.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(items, ensure_ascii=False))

filtered_items = []
for tour in items:
    if tour["min_rating"] >= 2500:
        filtered_items.append(tour)

with open("results/task1_filtered_min_rating_2500.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_items, ensure_ascii=False))

views_stats = calc_stats("views", items)
with open("results/task1_views_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(views_stats, ensure_ascii=False))

city_freq = calc_frequency("city", items)
with open("results/task1_city_freq.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(city_freq, ensure_ascii=False))

# Вторая задача

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import json


def handle_file(file_name):
    items = list()

    with open(file_name, encoding="utf-8") as file:
        text = ""
        for row in file.readlines():
            text += row

        site = BeautifulSoup(text, 'html.parser')
        products = site.find_all("div", attrs={'class': 'product-item'})

        for product in products:
            item = dict()
            item['id'] = product.a['data-id']
            item['link'] = product.find_all('a')[1]['href']
            item['img_url'] = product.find_all('img')[0]['src']
            item['title'] = product.find_all('span')[0].get_text().strip()
            item['price'] = int(product.price.get_text().replace("₽", "").replace(" ", "").strip())
            item['bonus'] = int(product.strong.get_text().replace("+ начислим ", "").replace(" бонусов", "").strip())
            props = product.ul.find_all("li")
            for prop in props:
                item[prop['type']] = prop.get_text().strip()

            items.append(item)

        return items


def calc_stats(col, items_lst):
    numbers = list(map(lambda x: x[col], items_lst))
    numeric_stats = {'sum': sum(numbers),
                     'avg': round(np.average(numbers), 2),
                     'min': min(numbers),
                     'max': max(numbers),
                     'std': round(np.std(numbers), 2),
                     }

    return numeric_stats


def calc_frequency(col, items_lst):
    frequency = {}

    for item in items_lst:
        if col in item:
            frequency[item[col]] = frequency.get(item[col], 0) + 1

    return frequency


items = []
for i in range(1, 29):
    file_name = f"tasks/task_2_var_54/{i}.html"
    items += handle_file(file_name)

items = sorted(items, key=lambda x: x['price'], reverse=True)

with open("results/task2_all.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(items, ensure_ascii=False))

filtered_items = []
for item in items:
    if item['bonus'] >= 2000:
        filtered_items.append(item)

with open("results/task2_filtered_min_bonus_2000.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_items, ensure_ascii=False))

price_stats = calc_stats("price", items)
with open("results/task2_price_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(price_stats, ensure_ascii=False))

ram_freq = calc_frequency("ram", items)
with open("results/task2_ram_freq.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(ram_freq, ensure_ascii=False))

# Третья задача

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")


def handle_file(file_name):
    with open(file_name, encoding="utf-8") as file:
        text = ""
        for row in file.readlines():
            text += row

    star = BeautifulSoup(text, 'lxml').star
    item = dict()
    item["name"] = star.find_all("name")[0].get_text().strip()
    item["constellation"] = star.find_all("constellation")[0].get_text().strip()
    item["spectral-class"] = star.find_all("spectral-class")[0].get_text().strip()
    item["radius"] = int(star.find_all("radius")[0].get_text().strip())
    item["rotation"] = star.find_all("rotation")[0].get_text().strip()
    item["age"] = star.find_all("age")[0].get_text().strip()
    item["distance"] = star.find_all("distance")[0].get_text().strip()
    item["absolute-magnitude"] = star.find_all("absolute-magnitude")[0].get_text().strip()

    return item


def calc_stats(col, items_lst):
    numbers = list(map(lambda x: x[col], items_lst))
    numeric_stats = {'sum': sum(numbers),
                     'avg': round(np.average(numbers), 2),
                     'min': min(numbers),
                     'max': max(numbers),
                     'std': round(np.std(numbers), 2),
                     }

    return numeric_stats


def calc_frequency(col, items_lst):
    frequency = {}

    for item in items_lst:
        if col in item:
            frequency[item[col]] = frequency.get(item[col], 0) + 1

    return frequency

items = []
for i in range(1, 501):
    file_name = f"tasks/task_3_var_54/{i}.xml"
    items.append(handle_file(file_name))

items = sorted(items, key=lambda x: int(x['radius']), reverse=True)

with open("results/task3_all.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(items, ensure_ascii=False))

filtered_items = []
for item in items:
    if item['radius'] >= 900000000:
        filtered_items.append(item)

with open("results/task3_filtered_min_radius_9e8.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_items, ensure_ascii=False))

radius_stats = calc_stats("radius", items)
with open("results/task3_radius_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(radius_stats, ensure_ascii=False))

constellation_freq = calc_frequency("constellation", items)
with open("results/task3_constellation_freq.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(constellation_freq, ensure_ascii=False))

# Четвёртая задача

In [ ]:
import json
from bs4 import BeautifulSoup
import numpy as np


def handle_file(file_name):
    items = list()

    with open(file_name, encoding="utf-8") as file:
        text = ""
        for row in file.readlines():
            text += row

        root = BeautifulSoup(text, 'xml')

        for clothing in root.find_all("clothing"):
            item = dict()
            for el in clothing.contents:
                if el.name is None:
                    continue
                elif el.name == "price" or el.name == "reviews":
                    item[el.name] = int(el.get_text().strip())
                elif el.name == "rating":
                    item[el.name] = float(el.get_text().strip())
                elif el.name == "new":
                    item[el.name] = el.get_text().strip() == "+"
                elif el.name == "exclusive" or el.name == "sporty":
                    item[el.name] = el.get_text().strip() == "yes"
                else:
                    item[el.name] = el.get_text().strip()

            items.append(item)

        return items


def calc_stats(col, items_lst):
    numbers = list(map(lambda x: x[col], items_lst))
    numeric_stats = {'sum': sum(numbers),
                     'avg': round(np.average(numbers), 2),
                     'min': min(numbers),
                     'max': max(numbers),
                     'std': round(np.std(numbers), 2),
                     }

    return numeric_stats


def calc_frequency(col, items_lst):
    frequency = {}

    for item in items_lst:
        if col in item:
            frequency[item[col]] = frequency.get(item[col], 0) + 1

    return frequency


items = []
for i in range(1, 101):
    file_name = f"tasks/task_4_var_54/{i}.xml"
    items += handle_file(file_name)

items = sorted(items, key=lambda x: x['price'], reverse=True)

with open("results/task4_all.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(items, ensure_ascii=False))

filtered_items = []
for item in items:
    if item['rating'] >= 4:
        filtered_items.append(item)

with open("results/task4_filtered_min_rating_4.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(filtered_items, ensure_ascii=False))

price_stats = calc_stats("price", items)
with open("results/task4_price_stats.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(price_stats, ensure_ascii=False))

category_freq = calc_frequency("category", items)
with open("results/task4_category_freq.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(category_freq, ensure_ascii=False))